# Day 5-Part 2: Seismic data

Seismic data are one of the most effective ways to image the subsurface. Although low resolution in comparison to well data (dam as opposed to cm), seismic data have a much larger coverage (km as opposed to m), and give an incomparable 3-D, and even 4-D (if data from the same area are acquired over time) visualization of the subsurface.

The most common format for seismic data is the SEG-Y format. Luckily in Python, the [segyio](https://github.com/equinor/segyio) library allows us to load and interact with `segy` data quite easily. In this notebook, we will use `segyio` to load and visualize a seismic cube.

First, we need to install `segyio`. If you have not done so, run the cell below:

In [ ]:
# run this cell if segyio is not installed

import sys
!{sys.executable} -m pip install segyio

We will use a subset of the F3 seismic cube, which is in the publicly available [F3 Demo 2020 project](https://terranubis.com/datainfo/F3-Demo-2020). The figure below (from OpendTect) shows the distribution of inlines, xlines, and TWT in the entire F3 cube. The figure is shown looking down and with North to the top. Two-way travel time (TWT) in ms increases downwards.

<img src="../figures/F3Survey.png" alt="F3Survey" width="600"/><br><br>

The segy file for this notebook is rather large (~370 MB), so I have placed it in a server. We can use the `urllib` library to retrieve the file and save it in the `data` directory. The code below is rather simple: we get the strings for the remote url (web address) and the local filename to save the data, and then we pass these strings to the `urllib.urlretrieve` method to download the file and save it locally. 

Run the cell below if you have not downloaded the file yet. Be patient, the file is relatively large and depending on your internet connection it may take some time to download:

In [ ]:
# run this cell if you don"t have the segy in the data directory

import os
# import request module of urllib
from urllib import request
# Define the remote file to retrieve
remote_url = "http://www.ux.uis.no/~nestor/Public/f3-dsmf.sgy"
# Define the local filename to save data
local_file = os.path.join("..", "data", "F3_seismic", "f3-dsmf.sgy")
# Download remote and save locally
request.urlretrieve(remote_url, local_file);

Next, we import the required libraries, including `segyio`:

In [ ]:
import os
import segyio
import matplotlib.pyplot as plt
import numpy as np

Now that we have sorted out these preliminaries, we can open the file using the `segyio.open` method. Also, from the filename `f`, we can extract the inlines, xlines, and TWT arrays using the `ilines`, `xlines`, and `samples` methods, respectively. The subtraction of the maximum and minimum values of these arrays gives us their ranges, and the `shape` of these arrays gives us their elements count:  

In [ ]:
filename = os.path.join("..", "data", "F3_seismic", "f3-dsmf.sgy") # segy file name

# open segy file and print inline, xline and twt ranges
with segyio.open(filename,"r") as f:
    print("Inline range:", np.amin(f.ilines), "-", np.amax(f.ilines), ", count:", f.ilines.shape[0])
    print("Xline range:", np.amin(f.xlines), "-", np.amax(f.xlines), ", count:", f.xlines.shape[0])
    print("TWT range [ms]:", np.amin(f.samples), "-", np.amax(f.samples), ", count", f.samples.shape[0])

So the file is a subset of the F3 cube and it is between inlines 200 and 600, xlines 500 and 1100, and TWT 200 and 1500 ms (check the figure above to understand what this means).

We can also get information about the step or increment between adjacent inlines, xlines, or time samples as follows:

In [ ]:
il_st = (np.amax(f.ilines) - np.amin(f.ilines)) / (f.ilines.shape[0] - 1) # inline step
xl_st = (np.amax(f.xlines) - np.amin(f.xlines)) / (f.xlines.shape[0] - 1) # xline step
twt_st = (np.amax(f.samples) - np.amin(f.samples)) / (f.samples.shape[0] - 1) # twt step
print("IL/XL/TWT steps:", il_st, ",", xl_st, ",", twt_st) # print steps

So, the inlines and xlines increment is 1, and the TWT increment is 4 ms.

Let's now read the seismic cube. To do this, we just use the `segyio.tools.cube` method, and give as input the `filename`. We also print the shape (or length) of the array along the inlines, xlines, and TWT dimensions:

In [ ]:
s_d = segyio.tools.cube(filename) # read seismic data
print("Survey IL/XL/TWT shape:", s_d.shape[0], "/", s_d.shape[1], "/", s_d.shape[2]) # print cube shape

The seismic cube `s_d` is a three-dimensional array made of 401 inlines (1st dimension), 601 xlines (2nd dimension), and 326 time samples (3rd dimension). In total, this array has 401 x 601 x 326 = 78'566,326 elements, which represent amplitude values at the intersections of the inlines, xlines, and TWT slices. This is a very large array!

Since we will use the seismic cube `s_d` to plot the data, we need to relate the indexes in the `s_d` array, to the actual values of the inlines, xlines, and TWT in the `f.ilines`, `f.xlines`, and `f.samples` arrays. This is important to label the plots correctly. We do this in the cell below. This code basically does the following:

1. Use a tick interval of 100 along the inlines or xlines axes, and 200 ms along the TWT axis.
2. Make two arrays: one for the positions (`pos`) of the ticks, and another for the labels (`lab`) of the ticks along the three axes: inlines (`il`), xlines (`xl`), and TWT (`twt`). Notice that the ticks positions are controlled by the shape of the `s_d` array, while the ticks labels are a function of the ticks positions and the starting inline, xline, and TWT, as well as their increments.

We'll use latter these arrays to set the locations and labels of the axes ticks:

In [ ]:
il_tl = 100 # step between tick labels for inlines
xl_tl = 100 # step between tick labels for xlines
twt_tl = 200 # step between tick labels for twt

il_pos = np.arange(0, s_d.shape[0], il_tl/il_st) # inlines ticks position
il_lab = il_pos * il_st + f.ilines[0] # inlines ticks labels
il_lab = il_lab.astype(int) # make integer inlines ticks labels
print("il ticks positions =", il_pos, "and labels =", il_lab)

xl_pos = np.arange(0, s_d.shape[1], xl_tl/xl_st) # xlines ticks position
xl_lab = xl_pos * xl_st + f.xlines[0] # xlines ticks labels
xl_lab = xl_lab.astype(int) # make integer xlines ticks labels
print("xl ticks positions =", xl_pos, "and labels =", xl_lab)

twt_pos = np.arange(0, s_d.shape[2], twt_tl/twt_st) # twt ticks positions
twt_lab = twt_pos * twt_st + f.samples[0] # twt ticks labels
twt_lab = twt_lab.astype(int) # make integer twt ticks labels
print("twt ticks positions =", twt_pos, "and labels =", twt_lab)

Now, we are ready to plot our data. We can use the code below to plot an inline. This code can be divided in the following parts: 

1. Choose an inline index `il_id`. The inline value for this index is `f.ilines[il_id]`.
2. Create a figure.
3. Plot the inline using the `ax.imshow` method, and passing the inline grid `s_d[il_id,:,:]`. Notice that to plot the inline correctly, we need to transpose `.T` this array, to have the xlines in the x axis, and the TWT in the y axis. In addition, we use a divergent red-blue (`RdBu`) colormap with specified minimum and maximum limits. This shows positive amplitudes in blue, and negative amplitudes in red.
4. Add a colorbar.
5. Plot the axes ticks based on the xlines and TWT ticks positions and labels from the cell above.
6. Label the axes and add a title to the plot. 

In [ ]:
# plot Inline

il_id = 100 # inline index

fig, ax = plt.subplots(figsize=(12,8)) # create figure

vm = np.percentile(s_d, 99) # 99 percentile of seismic data, nice limit for color bar

sim = ax.imshow(s_d[il_id,:,:].T, cmap="RdBu", vmin=-vm, vmax=vm) # plot inline

fig.colorbar(sim, label="Amplitude") # colorbar

plt.xticks(xl_pos, xl_lab) # xlines ticks

plt.yticks(twt_pos, twt_lab) # twt ticks

ax.set_xlabel("Xline") # xlabel
ax.set_ylabel("TWT [ms]") # y label
ax.set_title("Inline = " + str(f.ilines[il_id])); # title

This is quite nice, but we could do much better. Imagine placing a control in the figure that would allow us to set the inline and update the section interactively. This sounds quite challenging, but it's actually easy to do in Jupyter using `widgets`. These are just controls (e.g. sliders, textboxes, etc.) that allow changing variables in the code interactively.

We modify the code above to include a slider widget that changes the inline number and updates the section. In order to do this, we do the following:

1. Import the `ipywidgets` library as `widgets` to be able to use widgets.
2. Encapsulate the inline drawing code into a function called `inline_widget`, which takes as input the inline number
3. Set the initial value of the slider to the middle of the inline range.
4. Create the inline slider. Here we specify the minimum, maximum, and initial value of the slider. We also set `continuous update` to False, so that the section updates only when the mouse is released.
5. Finally, here is the magic: We call the method `ipywidgets.interact`, with the drawing function as the first input, and the value of the inline which is controlled by the slider as the second input.

Run the code. Move the inline slider, and see how the section updates to the new inline upon mouse release. This is pretty cool 😎

In [ ]:
# plot Inline

import ipywidgets as widgets

def inline_widget(inline):
    
    il_id = int ((inline - f.ilines[0]) / il_st) # inline index
    
    fig, ax = plt.subplots(figsize=(12,8)) # create figure

    sim = ax.imshow(s_d[il_id,:,:].T, cmap="RdBu", vmin=-vm, vmax=vm) # plot inline

    fig.colorbar(sim, label="Amplitude") # colorbar

    plt.xticks(xl_pos, xl_lab) # xlines ticks

    plt.yticks(twt_pos, twt_lab) # twt ticks

    ax.set_xlabel("Xline") # xlabel
    ax.set_ylabel("TWT [ms]") # y label
    ax.set_title("Inline = " + str(inline)); # title
    
# initial value of inline slider
il_id = int ((f.ilines.shape[0] - 1) * 0.5) # inline index
il = f.ilines[il_id] # inline

# Create inline slider
slider = widgets.IntSlider(min=np.amin(f.ilines), max=np.amax(f.ilines), 
                           value=il, continuous_update=False)    

# call inline function and make it interact with the slider
widgets.interact(inline_widget, inline=slider);

Let's do the same for plotting a xline. The `xline_widget` function is similar to the `inline_widget` function. From the input xline `xl`, we calculate the xline index `xl_id`, and use this index to plot the xline grid `s_d[:,xl_id,:]`. Again, we transpose `.T` this array to have the inlines in the x axis, and TWT in the y axis. The axes ticks are plotted based on the inlines and TWT ticks positions and labels arrays. Finally, the way we create the xline slider, and its interaction with the function is similar to the code above.

Run the code. Move the xline slider and see how the section updates to the new xline upon mouse release.

In [ ]:
# plot Xline

def xline_widget(xline):
    
    xl_id = int ((xline - f.xlines[0]) / xl_st) # xline index

    fig, ax = plt.subplots(figsize=(8,6)) # create figure

    sim = ax.imshow(s_d[:,xl_id,:].T, cmap="RdBu", vmin=-vm, vmax=vm) # plot xline

    fig.colorbar(sim, label="Amplitude") # colorbar

    plt.xticks(il_pos, il_lab) # inlines ticks

    plt.yticks(twt_pos, twt_lab) # twt ticks

    ax.set_xlabel("Inline") # xlabel
    ax.set_ylabel("TWT [ms]") # y label
    ax.set_title("Xline = " + str(xline)); # title 
    
# initial value of xline slider
xl_id = int ((f.xlines.shape[0] - 1) * 0.5) # xline index
xl = f.xlines[xl_id] # xline

# make xline slider
slider = widgets.IntSlider(min=np.amin(f.xlines), max=np.amax(f.xlines), 
                          value=xl, continuous_update=False)

# call xline function and make it interact with the slider
widgets.interact(xline_widget, xline=slider);

Finally, let's plot a TWT slice. In the `twt_widget` function, from the input TWT value `twt`, we calculate the TWT index `twt_id`. We then plot the TWT grid `s_d[:,:,twt_id]`. In this case, we don't need to transpose the grid, since the xlines plot along x, and the inlines along y, which is what we want. The other parts of the function are similar to the previous functions. There is however a minor detail: in order to have the inlines increasing to the north (top of the figure), we need to invert the y axis `ax.invert_yaxis`. The creation of the TWT slider and interaction with the function is similar to the previous codes for inlines and xlines.

Run the code and move the TWT slider to change the time slice:

In [ ]:
# plot time slice

def twt_widget(twt):
    
    twt_id = int ((twt - f.samples[0]) / twt_st) # time slice index

    fig, ax = plt.subplots(figsize=(12,6)) # create figure

    sim = ax.imshow(s_d[:,:,twt_id], cmap="RdBu", vmin=-vm, vmax=vm) # plot time slice

    fig.colorbar(sim, label="Amplitude") # colorbar

    plt.xticks(xl_pos, xl_lab) # xlines ticks

    plt.yticks(il_pos, il_lab) # inline ticks

    ax.set_xlabel("Xline") # xlabel
    ax.set_ylabel("Inline") # y label
    ax.set_title("Time slice = " + str(twt) + " ms") # title
    ax.invert_yaxis() # invert y axis
    
# initial value of twt slider
twt_id = int ((f.samples.shape[0] - 1) * 0.5) # time slice index
twt = f.samples[twt_id] # time slice

# make twt slider
slider = widgets.IntSlider(min=np.amin(f.samples), max=np.amax(f.samples), 
                           value=twt, step=twt_st, continuous_update=False)  

# call twt function and make it interact with the slider
widgets.interact(twt_widget, twt=slider);

What about if we want to plot the three orthogonal slices in one graph, and also show in the slice where the other two are located? The code below will do it:

In [ ]:
# Plot all three in a graph and show intersecting slices

def seismic_widget(inline, xline, twt):
    
    fig, ax = plt.subplots(2,2,figsize=(14,12)) # create figure
    
    # id"s
    # inline
    il_id = int ((inline - f.ilines[0]) / il_st) # inline index
    # xline
    xl_id = int ((xline - f.xlines[0]) / xl_st) # xline index
    # time slice
    twt_id = int ((twt - f.samples[0]) / twt_st) # time slice index
    
    # inline
    data = s_d[il_id,:,:]
    
    ax[0,0].imshow(data.T, cmap="RdBu", vmin=-vm, vmax=vm, aspect="auto") # plot inline
    ax[0,0].axhline(twt_id,color="black",linestyle="--",linewidth=2) # mark twt slice
    ax[0,0].axvline(xl_id,color="black",linestyle="--",linewidth=2) # mark xline slice

    ax[0,0].set_xticks(xl_pos) # xlines ticks positions
    ax[0,0].set_xticklabels(xl_lab) # xlines ticks labels
    ax[0,0].set_yticks(twt_pos) # twt ticks positions
    ax[0,0].set_yticklabels(twt_lab) # twt ticks labels
    
    ax[0,0].set_xlabel("Xline") # xlabel
    ax[0,0].set_ylabel("TWT [ms]") # y label
    ax[0,0].set_title("Inline = " + str(inline)) # title
    
    # xline
    data = s_d[:,xl_id,:]

    ax[0,1].imshow(data.T, cmap="RdBu", vmin=-vm, vmax=vm, aspect="auto") # plot xline
    ax[0,1].axhline(twt_id,color="black",linestyle="--",linewidth=2) # mark twt slice
    ax[0,1].axvline(il_id,color="black",linestyle="--",linewidth=2) # mak inline slice

    ax[0,1].set_xticks(il_pos) # ilines ticks positions
    ax[0,1].set_xticklabels(il_lab) # ilines ticks labels
    ax[0,1].set_yticks(twt_pos) # twt ticks positions
    ax[0,1].set_yticklabels(twt_lab) # twt ticks labels

    ax[0,1].set_xlabel("Inline") # xlabel
    ax[0,1].set_ylabel("TWT [ms]") # y label
    ax[0,1].set_title("Xline = " + str(xline)) # title
    
    # time slice
    data = s_d[:,:,twt_id]

    sim = ax[1,0].imshow(data, cmap="RdBu", vmin=-vm, vmax=vm, aspect="auto") # plot time slice
    ax[1,0].axhline(il_id,color="black",linestyle="--",linewidth=2) # mark iline slice
    ax[1,0].axvline(xl_id,color="black",linestyle="--",linewidth=2) # mark xline slice
    
    fig.colorbar(sim, label="Amplitude", ax=ax[1,1], location="left") # colorbar
    
    ax[1,0].set_xticks(xl_pos) # xlines ticks positions
    ax[1,0].set_xticklabels(xl_lab) # xlines ticks labels
    ax[1,0].set_yticks(il_pos) # ilines ticks positions
    ax[1,0].set_yticklabels(il_lab) # ilines ticks labels

    ax[1,0].set_xlabel("Xline") # xlabel
    ax[1,0].set_ylabel("Inline") # y label
    ax[1,0].set_title("Time slice = " + str(twt) + " ms") # title
    ax[1,0].invert_yaxis() # invert y axis, inlines increase north
    
    ax[1,1].set_visible(False)

# initial value of inline slider
il_id = int ((f.ilines.shape[0] - 1) * 0.5) # inline index
il = f.ilines[il_id] # inline

# Create inline slider
il_slider = widgets.IntSlider(min=np.amin(f.ilines), max=np.amax(f.ilines), \
                           value=il, continuous_update=False)    

# initial value of xline slider
xl_id = int ((f.xlines.shape[0] - 1) * 0.5) # xline index
xl = f.xlines[xl_id] # xline

# make xline slider
xl_slider = widgets.IntSlider(min=np.amin(f.xlines), max=np.amax(f.xlines), \
                          value=xl, continuous_update=False)

# initial value of twt slider
twt_id = int ((f.samples.shape[0] - 1) * 0.5) # time slice index
twt = f.samples[twt_id] # time slice

# make twt slider
twt_slider = widgets.IntSlider(min=np.amin(f.samples), max=np.amax(f.samples), \
                           value=twt, step=twt_st, continuous_update=False)  

# call inline function and make it interact with the sliders
widgets.interact(seismic_widget, inline=il_slider, xline=xl_slider, twt=twt_slider);

That's it. We have made a Jupyter notebook that allows slicing and visualizing a seismic cube!